<a href="https://colab.research.google.com/github/mehrarishab/image_classification/blob/main/InputPipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!ls /content/drive/MyDrive/datasets

 covid_dataset	 COVID_IEEE   jobs.json   potato  'Untitled Jam.gjam'


In [3]:
import tensorflow as tf

In [7]:
ds_path = '/content/drive/MyDrive/datasets/potato'

In [19]:
image_ds = tf.data.Dataset.list_files(ds_path + '/*/*', shuffle=True)

# Shuffle the Tensor Dataset with Buffer Size of 200
image_ds = image_ds.shuffle(200)

# Batch the Tensor Dataset with Batch Size of 32
# image_ds = image_ds.batch(32)

for img_file_loc in image_ds.take(10):
  print(img_file_loc.numpy())

<class 'bytes'>
b'/content/drive/MyDrive/datasets/potato/Late_blight/63490f84-88b0-4105-bb76-23990e98c692___RS_LB 3041.JPG'
<class 'bytes'>
b'/content/drive/MyDrive/datasets/potato/Late_blight/dd08ad51-110b-4de0-bfbb-862e53a6165c___RS_LB 4542.JPG'
<class 'bytes'>
b'/content/drive/MyDrive/datasets/potato/Early_blight/4af8e9e9-077d-4c7b-859d-626f74d3b1ff___RS_Early.B 7738.JPG'
<class 'bytes'>
b'/content/drive/MyDrive/datasets/potato/Late_blight/b11189c8-ab6f-4d69-9ffa-e52749d2a3a5___RS_LB 4537.JPG'
<class 'bytes'>
b'/content/drive/MyDrive/datasets/potato/Late_blight/f2676f08-798e-4291-8aa2-0655ca54cb07___RS_LB 4292.JPG'
<class 'bytes'>
b'/content/drive/MyDrive/datasets/potato/Late_blight/b0e6c386-35dc-4982-acc6-f393c7f58c67___RS_LB 5036.JPG'
<class 'bytes'>
b'/content/drive/MyDrive/datasets/potato/Late_blight/cba4be0b-2554-4a9e-8dd0-94cbb2467388___RS_LB 3335.JPG'
<class 'bytes'>
b'/content/drive/MyDrive/datasets/potato/Early_blight/07953ca1-8935-449f-b338-4357ed683b2d___RS_Early.B 6815.J

In [25]:
# Generate Label from Image Path
import os

def get_label(file_path):
  parts = tf.strings.split(file_path,'/')
  return parts[-2]


def process_image(file_path):
  label = get_label(file_path)
  img = tf.io.read_file(file_path)
  img = tf.image.decode_jpeg(img, channels=3)
  img = tf.image.resize(img,[224,224])
  img = tf.cast(img, tf.float32)
  img /= 255
  return img, label

In [ ]:
process_image('/content/drive/MyDrive/datasets/potato/Late_blight/2c2b39ed-75f9-49f6-a28e-36a2cb608297___RS_LB 2794.JPG')

In [31]:
# Split Image Dataset into Train and Test
image_count = int(len(image_ds) * 0.8)
train_ds = image_ds.take(image_count)
test_ds = image_ds.skip(image_count)

print('Size of Training Dataset: ' ,len(train_ds))
print('Size of Test Dataset: ', len(test_ds))

Size of Training Dataset:  1721
Size of Test Dataset:  431


In [32]:
# Process the entire Training and Test Data Sets using map function

train_ds = train_ds.map(process_image)
test_ds = test_ds.map(process_image)

In [33]:
for image, label in train_ds.take(5):
    print("****Image: ",image.numpy()[0][0])
    print("****Label: ",label.numpy())

****Image:  [0.80864346 0.8008003  0.82040817]
****Label:  b'Early_blight'
****Image:  [0.75152063 0.7593638  0.81034416]
****Label:  b'Early_blight'
****Image:  [0.6291116  0.6055822  0.62126845]
****Label:  b'Early_blight'
****Image:  [0.49581832 0.4605242  0.48013204]
****Label:  b'Late_blight'
****Image:  [0.63639456 0.6285514  0.6795318 ]
****Label:  b'Early_blight'


In [44]:
# Check the Distribution of Classes in Train & Test Dataset

class_count = {'Early_blight': 0 , 'Late_blight': 0, 'healthy':0}

for image, label in train_ds:
  classname = label.numpy().decode('utf-8')
  class_count[classname] += 1

print(class_count)


{'Early_blight': 785, 'Late_blight': 815, 'healthy': 121}
